In [17]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.abspath(r"..\help-desk-agent-flask-app\.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [18]:
from langchain_google_genai import GoogleGenerativeAI
#Initialize a Gemini model
gemini_llm = GoogleGenerativeAI(model="gemini-2.0-flash-lite")
#Either one or both can be used with the same interface
response = gemini_llm.invoke("tell me a joke about light bulbs!")
print(response)

Why did the light bulb get a promotion?

Because it was always bright!

